In [194]:
import numpy as np

##### Constants (Peacemaker)

In [160]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Partition Functions

### Translational Partition Function (calculate_lnqtrans)

$q_{\text{trans}} = \frac{V-b_{xv}V_{\text{excl}}}{\Lambda^3}$ </p>
$ \ln(q_{\text{trans}}) = \ln(V-V_{\text{excl}}\cdot b_{xv}) -3\ln(\Lambda)$ </p>
$\Lambda = \sqrt{\frac{h^2}{2\pi mkT}}$ </p>

In [161]:
# Calculates the partition function for one cluster

def calculate_lnqtrans(lnqtrans, bxv, temp, v, mass, v_excl):
    # lnqtrans  : input/output  - ln of the translational partition function
    # bxv       : input         - van der Waals scaling factor 
    # temp      : input         - temperature
    # v         : input         - volume of the cluster
    # mass      : input         - mass of the cluster
    # v_excl    : input         - excluded volume
    
    m = mass*amu
    lambda_ = planck/np.sqrt(2.0*pi*m*kb*temp)
    
    lnqtrans = np.log(v-bxv*v_excl) - 3.0*np.log(lambda_)
    return lnqtrans

#### Tests

In [162]:
lnqtrans = 0
bxv = 0.8
temp = 298.0
v = 2.0
mass = 0.01
v_excl = 0.1
lnqtrans = calculate_lnqtrans(lnqtrans, bxv, temp, v, mass, v_excl)
print(lnqtrans)

62.78833143890738


### Vibrational Partition Fuction (calculate_lnqvib)

Harmonic oscillator </p>
$q_{\text{vib}} = \frac{\exp(-\Theta^{\text{vib}}/2T)}{1-\exp(-\Theta^{\text{vib}}/T)}$ </p>
$\ln(q_{\text{vib}}) = -\frac{\Theta^{\text{vib}}}{2T} - \ln(1-\exp(-\Theta^{\text{vib}}/T))$ </p>
$\Theta^{\text{vib}} = \frac{h\nu}{k_B}$ </p>
$\frac{h\nu}{k_BT} = \frac{hc}{k_BT} \cdot \~{v}$ </p>

Morse oscillator (Anharmonic effects in the quantum cluster equilibrium method -2017) </p>
$E_n = hc\nu [(n+ \frac{1}{2})-\mathcal{X}_e(n+ \frac{1}{2})^2]~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ 
$\mathcal{X}_e$ : the anharmonicity constant </p>
$q_i^{\text{morse}} = \prod_{\mu=1}^{\mu_{\text{max}}} \sum_{n=0}^{n^{\text{max}}} \exp(-(n+ \frac{1}{2})) \frac{\Theta_{i,\mu}^{\text{vib}}}{T}+\mathcal{X}_e(n+ \frac{1}{2})^2\frac{\Theta_{i,\mu}^{\text{vib}}}{T}~~~~~~$  $\mu$ : vibrational modes </p> 
$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ 
$n^{\text{max}}$ : highest vibrational energy level before dissociation for the respective mode </p>

$\Rightarrow$ Strekalov $\Rightarrow$ </p>
$q_i^{\text{morse}} = \prod_{\mu=1}^{\mu_{\text{max}}} \frac{1}{2 \cdot \sinh(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T})} \exp(\mathcal{X}\frac{\Theta_{i,\mu}^{\text{vib}}}{T}(\frac{1}{4}+\frac{1}{2 \cdot \sinh(\frac{\Theta_{\mu}^{\text{vib}}}{2T})^2}))$ </p>
$ ln(q_i^{\text{morse}}) = \sum_{\mu=1}^{\mu_{\text{max}}} \Big[-\ln(2 \cdot \sinh(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T})) + \mathcal{X}\frac{\Theta_{i,\mu}^{\text{vib}}}{T}(\frac{1}{4}+\frac{1}{2 \cdot \sinh(\frac{\Theta_{\mu}^{\text{vib}}}{2T})^2})\Big]$ </p>

Free rotator (Supramolecular Binding Thermodynamics by Dispersion-Corrected Density Functional Theory - Grimme 2012) </p>

$\mu = \frac{h}{8 \pi^2 \nu}$ </p>
$\mu' = \frac{\mu B_{\text{av}}}{\mu + B_{\text{av}}}~~~~~~~~~~~~~~~~~~~~~~~$ Moment of inertia of the cluster with frequency $\omega$</p>
$B_{\text{av}} = \sum_{n} \frac{I_n}{N}~~~~~~~~~~~~~~~~~~~$ Avarage moment of inertia of the cluster</p>
$T_{\text{rot}} = \frac{\hbar^2}{2 k_{\text{B}} \mu'}$ </p>
$q^{\text{rot}} = \frac{1}{\sigma} \sqrt{\frac{\pi T}{T_{\text{rot}}} }~~~~~~~~~~~~~~~~~$ $\sigma$ rotational symmetry nuber </p>
$\ln(q^{\text{rot}}) = \ln(\frac{1}{\sigma}) + \frac{1}{2} \ln(\frac{\pi T}{T_{\text{rot}}})$ </p>
$w(\omega) = \frac{1}{1+(\text{rotor cutoff}/\nu)^4}$

$q_{\text{vib,fr}} = w(\omega) \cdot q_{\text{ho}} + (1-w(\omega)) \cdot q_{\text{fr}}~~~$ or 
$~~~q_{\text{vib}} = q_{\text{aho}}$  </p>

In [163]:
# Calculates the vibrational partition function for one cluster

def calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const):
    # lnqvib       : input/output  - ln of the vibrational partition function
    # temp         : input         - temperature
    # freqs        : input         - vibrational frequencies
    # inertia      : input         - moments of inertia (number of moments of inertia depends on shape of the cluster)
    # sigma        : input         - symmetry number of the cluster
    # rotor_cutoff : input         - cutoff frequency for the free rotor approximation
    # anharm_const : input         - anharmonicity constant

    # hc/k
    factor = planck * speed_of_light * 100 / kb 
    # Avarage moment of inertia
    Bav = sum(inertia)/len(inertia) * amu * 1.0e-20

    # Initialize variables
    lnq_aho = 0.0
    lnq_ho = 0.0
    lnq_fr = 0.0
    lnqvib = 0.0
    w = 1.0
    
    for i in range(len(freqs)):
        # Case 1: Harmonic oscillator
        if (anharm_const == 0.0):
            t_vib = factor * freqs[i]
            lnq_ho = -t_vib / (2.0 * temp) - np.log(1 - np.exp(-t_vib / temp))

        
            # Case 1.1: Free rotor
            if (rotor_cutoff > 0.0):
                mu = planck/(8.0 * pi**2 * freqs[i] * speed_of_light * 100.0)
                mu_prime = mu*Bav/(mu+Bav)
                trot = hbar**2/(2.0*kb*mu_prime)
                lnq_fr = np.log(1.0/sigma * (np.sqrt(pi*temp/trot)))
                w = 1.0/(1+(rotor_cutoff/freqs[i])**4)
                
        # Case 2: Anharmonic oscillator
        else:
            t_vib = factor * freqs[i]
            # (Logarithm laws)
            lnqvib = lnqvib - np.log(2.0 * np.sinh(t_vib / (2.0 * temp))) + anharm_const * (t_vib / temp) * (0.25 + 1.0 / (2.0 * np.sinh(t_vib / (2.0 * temp))**2))
        
        lnqvib += w * lnq_ho + (1.0 - w) * lnq_fr
    return lnqvib

##### Tests

In [164]:
# Case 1 : Harmonic oscillator
lnqvib = 0.0
temp = 315.00
freqs = [11.4, 25.6, 68.7, 134.9, 3555.9]
inertia = [212.4, 212.4]
sigma = 2
rotor_cutoff = 0
anharm_const = 0.0
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

result -1.3967910870430646


In [165]:
# Case 1.1 : Harmonic oscillator with free rotator
lnqvib = 0.0
temp = 298.00
freqs = [5.0, 648.0, 1000.0, 3555.7]
inertia = [3.2, 5.9, 89.3, 1.0]
sigma = 1
rotor_cutoff = 3000.0
anharm_const = 0.0
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

# Case 1.1 : Harmonic oscillator with free rotator
lnqvib = 0.0
temp = 298.00
freqs = [100.0, 200.0, 300.0]
inertia = [1.0, 1.0, 1.0]
sigma = 1
rotor_cutoff = 3000.0
anharm_const = 0.0
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

result -3.855446885285766
result 1.7676272441463634


In [166]:
# Case 2 : Anharmonic oscillator
lnqvib = 0.0
temp = 298.00
freqs = [180.0, 270.0, 3550.0]
inertia = [1.0, 3.0, 1.0]
sigma = 1
rotor_cutoff = 3000.0
anharm_const = 0.6
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

# Case 2 : Anharmonic oscillator
lnqvib = 0.0
temp = 315.00
freqs = [5.0, 648.0, 1000.0, 3555.7]
inertia = [38.6, 43.9, 112.9]
sigma = 1
rotor_cutoff = 0
anharm_const = 0.34
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

result -3.8012813311299443
result 23.9041034243484


### Rotational Partition Function (calculate_lnqrot)

Atom </p>
$q_{\text{rot}} = 1$
$\ln(q_{\text{rot}}) = 0$ </p> </br>

Linear cluster </p>
$q_{\text{rot}} = \frac{1}{\sigma}  \cdot \frac{T}{\Theta^{\text{rot}}}$  </p>
$\ln(q_{\text{rot}}) = \ln(\frac{1}{\sigma}) + \ln(\frac{T}{\Theta^{\text{rot}}})$ </p> </br>

Nonlinear cluster </p>
$q_{\text{rot}} = \frac{1}{\sigma}  \cdot \sqrt{\frac{\pi T^3}{\Theta^{\text{rot}}_1\Theta^{\text{rot}}_2\Theta^{\text{rot}}_3}}$  </p>
$\ln(q_{\text{rot}}) = \ln(\frac{1}{\sigma}) + \ln(\sqrt{\frac{\pi T^3}{\Theta^{\text{rot}}_1\Theta^{\text{rot}}_2\Theta^{\text{rot}}_3}})$ </p> </br>

$T_{\text{rot}} = \Theta^{\text{rot}} = \frac{\hbar^2}{2 k_{\text{B}} I}$ </p>

In [167]:
# Calculates the rotational partition function for one cluster

def calculate_lnrot(lnq, temp, inertia, sigma, atom, linear):
    # lnq     : input/output  - ln of the rotational partition function
    # temp    : input         - temperature
    # inertia : input         - moments of inertia (number of moments of inertia depends on shape of the cluster)
    # sigma   : input         - symmetry number of the cluster
    # atom    : input         - boolean, true if cluster is atom
    # linear  : input         - boolean, true if cluster is linear
    
    
    # Case 1: Cluster is an atom
    if (atom):
        lnq = 0.0
    # Case 2: Cluster is not an atom
    else:
        lnq = 1.0
        for i in range(len(inertia)):
            # Calculate the rotational temperature
            trot = hbar**2/(2.0*kb*inertia[i]*amu*1.0e-20)
            # Calculate the part of the rotational partition function 
            # that is equal for linear and non-linear clusters
            lnq *= temp/trot
            
        # Case 2.1: Linear cluster
        # The inertia array contains two equal moments of inertia
        # The square root is taken to account for the degeneracy of the rotational levels
        if (linear):
            lnq = 1/sigma * np.sqrt(lnq)
            
        # Case 2.2: Non-linear cluster
        # The inertia array contains three moments of inertia
        else:
            lnq = 1/sigma * np.sqrt(pi*lnq)
            #print('lnq n',lnq)
                
        lnq = np.log(lnq)
            
    return lnq
            

#### Tests

In [168]:
# Case 1: Cluster is an atom
lnq = 0.0
temp = 298.00
inertia = [1.0, 1.0, 1.0]
sigma = 1
atom = True
linear = False
lnq = calculate_lnrot(lnq, temp, inertia, sigma, atom, linear)
print('result',lnq)

result 0.0


In [169]:
# Case 2.1: Linear cluster
lnq = 0.0
temp = 298.00
inertia = [37.4, 37.4] 
sigma = 1
atom = False
linear = True
lnq = calculate_lnrot(lnq, temp, inertia, sigma, atom, linear)
print('result',lnq)

result 6.130167446075413


In [170]:
# Case 2.2: Non-linear cluster
lnq = 0.0
temp = 298.00
inertia = [21.2, 43.2, 70.0]
sigma = 2
atom = False
linear = False
lnq = calculate_lnrot(lnq, temp, inertia, sigma, atom, linear)
print('result',lnq)

result 9.17613133433584


### Electronic Partition Function (calculate_lnqelec)

$q_{\text{elec}} = g \cdot \exp(-\frac{E_{\text{elec}}}{kT})$ </p>
$\ln(q_{\text{elec}}) = \ln(g) - \frac{E_{\text{elec}}}{kT}$ </p>

In [171]:
# Calculates the electronic partition function for one cluster

def calculate_lnqelec(lnqelec, temp, energy):
    # lnq  : input/output  - ln of the electronic partition function
    # temp : input         - temperature
    
    lnqelec = -1.0/avogadro * energy/(kb*temp) * 1000.0 # kJ/mol to J/mol (energy is in kJ/mol)
    return lnqelec

#### Tests

In [172]:
lnqelec = 0.0
temp = 318.15
energy = -28.18
lnqelec = calculate_lnqelec(lnqelec, temp, energy)
print('result',lnqelec)

result 10.653072928364542


### Mean Field partition function (calculate_lnqint)

$ \epsilon_{\text{mf}} = - a_{mf} \cdot \frac{N}{V}$ </p>
$ N = \text{number of components} \cdot  N_{\text{tot}}$ </p>
$ N_{\text{tot}} = \sum^{N_{\text{components}}} \text{monomer amount of the component (\%)} \cdot N_\text{A} = N_\text{A}$ </p>
$ ln(q_{\text{int}}) = -\frac{\epsilon_{\text{mf}}}{K_{\text{B}}T} $ </p>

In [173]:
# Calculates the mean field partition function for one cluster

def calculate_lnqint(lnqint, amf, temp, v, composition, ntot):
    # lnqint       : input/output  - ln of the mean field partition function
    # amf          : input         - van der Waals pressure scaling factor
    # temp         : input         - temperature
    # v            : input         - volume of the cluster
    # composition  : input         - composition 
    # ntot         : input         - total number of particles
    
    e_mf = -amf * sum(composition) * ntot / v
    lnqint = -e_mf / (kb * temp)    
    
    return lnqint

#### Tests

In [174]:
lnqint = 0.0
amf = 1.0e-48
temp = 450.0
v = 3.5e-3
composition = [1, 6, 2]
ntot = (0.3 + 0.5 + 0.2) * avogadro
lnqint = calculate_lnqint(lnqint, amf, temp, v, composition, ntot)
print('result',lnqint)

result 0.2492468468447588


# Temp. derivatives of the partition functions

### Translational Partition Function Derivative (calculate_dlnqtrans)

$ \ln(q_{\text{trans}}) = -3\ln(\Lambda) + \ln(V-V_{\text{excl}}\cdot b_{xv}) $ </p>
$\Lambda = \sqrt{\frac{h^2}{2\pi mkT}}$ </p> 

*$b_{xv}$ scales linear with the temperature with the linear dependence parameter $b_{xv, \text{temp}}$ 
and thus we know: $\frac{db_{xv}}{dT} = b_{xv, \text{temp}}$*</p>


$ \frac{d\ln(q_{\text{trans}})}{dT} = \frac{3}{2T} + \frac{-V_{\text{excl}}\cdot b_{xv, \text{temp}}}{V-V_{\text{excl}}\cdot b_{xv}}$ </p>

In [175]:
# Calculates the temp. derivative of the ln of the translational partition function for one cluster

def calculate_dlnqtrans(dlnqtrans, bxv, bxv_temp, temp, v, v_excl):
    # dlnqtrans  : input/output  - derivative of ln of the translational partition function
    # bxv        : input         - van der Waals scaling factor
    # bxv_temp   : input         - derivative of van der Waals scaling factor with respect to temperature
    # temp       : input         - temperature
    # v          : input         - volume of the cluster
    # v_excl     : input         - excluded volume
    
    dlnqtrans = 1.5 / temp + (v_excl * bxv_temp) / (v_excl * bxv - v)
    return dlnqtrans

#### Tests

In [176]:
dlnqtrans = 0.0
bxv = 0.8
bxv_temp = 0.9
temp = 431.0
v = 2.5e-3
v_excl = 1e-4

dlnqtrans = calculate_dlnqtrans(dlnqtrans, bxv, bxv_temp, temp, v, v_excl)
print('result',dlnqtrans)

result -0.033709804222354325


In [177]:
dlnqtrans = 0.0
bxv = 2
bxv_temp = 1.3
temp = 315.2
v = 2.1
v_excl = 1

dlnqtrans = calculate_dlnqtrans(dlnqtrans, bxv, bxv_temp, temp, v, v_excl)
print('result',dlnqtrans)

result -12.995241116751258


### Vibrational Partition Function Derivative (calculate_dlnqvib)

Harmonic oscillator </p>
$\ln(q_{\text{vib}}) = -\frac{\Theta^{\text{vib}}}{2T} - \ln(1-\exp(-\Theta^{\text{vib}}/T))$ </p>
$\frac{d\ln(q_{\text{vib}})}{dT} = \frac{\Theta^{\text{vib}}}{2T^2} + \frac{\Theta^{\text{vib}}}{T^2(\exp(\Theta^{\text{vib}}/T)-1)}$

Anharmonic oscillator </p>
$ ln(q_i^{\text{morse}}) = \sum_{\mu=1}^{\mu_{\text{max}}} \Big[-\ln(2 \cdot \sinh(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T})) + \mathcal{X}\frac{\Theta_{i,\mu}^{\text{vib}}}{T}(\frac{1}{4}+\frac{1}{2 \cdot \sinh(\frac{\Theta_{\mu}^{\text{vib}}}{2T})^2})\Big]$ </p>
$ \frac{d\ln(q_i^{\text{morse}})}{dT} = \sum_{\mu=1}^{\mu_{\text{max}}} \Bigg[ \dfrac{\Theta_{i,\mu}^{\text{vib}}\cosh\left(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T}\right)}{2\sinh\left(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T}\right)\,T^2}-\dfrac{\Theta_{i,\mu}^{\text{vib}}x\cdot\left(\frac{1}{2\sinh^2\left(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T}\right)}+\frac{1}{4}\right)}{T^2}+\dfrac{{\Theta_{i,\mu}^{\text{vib}}}^2x\cosh\left(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T}\right)}{2\sinh^3\left(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T}\right)\,T^3} \Bigg]$ 

Free rotator </p>
$\ln(q^{\text{rot}}) = \ln(\frac{1}{\sigma}) + \frac{1}{2} \ln(\frac{\pi T}{T_{\text{rot}}})$ </p>
$\frac{d\ln(q^{\text{rot}})}{dT} = \frac{1}{2T}$

In [178]:
# Calculates the temp. derivative of the ln of the vibrational partition function for one cluster

def calculate_dlnqvib(dlnqvib, temp, freqs, rotor_cutoff, anharm_const):
    # dlnqvib       : input/output - temp. derivative of the ln of the vibrational partition function
    # temp         : input         - temperature
    # freqs        : input         - vibrational frequencies
    # inertia      : input         - moments of inertia (number of moments of inertia depends on shape of the cluster)
    # sigma        : input         - symmetry number of the cluster
    # rotor_cutoff : input         - cutoff frequency for the free rotor approximation
    # anharm_const : input         - anharmonicity constant

    # hc/k
    factor = planck * speed_of_light * 100 / kb 

    # Initialize variables
    dlnq_ho = 0.0
    dlnq_fr = 0.0
    dlnqvib = 0.0
    w = 1.0
    
    for i in range(len(freqs)):
        t_vib = factor * freqs[i]
        
        # Case 1: Harmonic oscillator and free rotor (if rotor_cutoff = 0.0 then w = 1.0) -> no if statement needed
        if (anharm_const == 0.0):
            dlnq_ho = t_vib / (2.0 * temp**2) + (t_vib / (temp**2 * (np.exp(t_vib / temp) - 1)))

            # Case 1.1 : HO and Free rotor              
            dlnq_fr = 1.0 / (2.0 * temp)
            w = 1.0/(1+(rotor_cutoff/freqs[i])**4)
                
        # Case 2: Anharmonic oscillator
        else:
            dlnqvib = dlnqvib + (t_vib * np.cosh(t_vib/(2.0*temp))) / (2.0 * np.sinh(t_vib/(2.0*temp)) * temp**2) \
                              - (t_vib * anharm_const * (0.25 + 1.0 / (2.0 * np.sinh(t_vib / (2.0 * temp))**2))) / temp**2 \
                              + (t_vib**2 * anharm_const * np.cosh(t_vib/(2.0*temp))) / (2.0 * np.sinh(t_vib/(2.0*temp))**3 * temp**3)    
            
        dlnqvib += w * dlnq_ho + (1.0 - w) * dlnq_fr
    return dlnqvib

#### Tests

In [179]:
# Case 1 : Harmonic oscillator
dlnqvib = 0.0
temp = 315.00
freqs = [11.4, 25.6, 68.7, 134.9, 3555.9]
rotor_cutoff = 0
anharm_const = 0.0

dlnqvib = calculate_dlnqvib(dlnqvib, temp, freqs, rotor_cutoff, anharm_const)
print('result',dlnqvib)

result 0.03860909908840071


In [180]:
# Case 1.1 : Harmonic oscillator, rotor_cutoff > 0
dlnqvib = 0.0
temp = 315.00
freqs = [11.4, 25.6, 68.7, 134.9, 3555.9]
rotor_cutoff = 2398.0
anharm_const = 0.0

dlnqvib = calculate_dlnqvib(dlnqvib, temp, freqs, rotor_cutoff, anharm_const)
print('result',dlnqvib)

result 0.02798357483860576


In [181]:
# Case 2 : Anharmonic oscillator
dlnqvib = 0.0
temp = 315.00
freqs = [11.4, 25.6, 68.7, 134.9, 3555.9, 10000.0]
rotor_cutoff = 0
anharm_const = 3.5

dlnqvib = calculate_dlnqvib(dlnqvib, temp, freqs, rotor_cutoff, anharm_const)
print('result',dlnqvib)

result 0.6617367002746789


### Rotational Partition Function Derivative (calculate_dlnqrot)

Atom </p>
$\ln(q_{\text{rot}}) = 0$ </p> 
$\frac{d\ln(q_{\text{rot}})}{dT} = 0$ </p> </br>

Linear cluster </p>
$\ln(q_{\text{rot}}) = \ln(\frac{1}{\sigma}) + \ln(\frac{T}{\Theta^{\text{rot}}})$ </p> 
$\frac{d\ln(q_{\text{rot}})}{dT} = \frac{1}{T}$ </p> </br>

Nonlinear cluster </p>
$\ln(q_{\text{rot}}) = \ln(\frac{1}{\sigma}) + \ln(\sqrt{\frac{\pi T^3}{\Theta^{\text{rot}}_1\Theta^{\text{rot}}_2\Theta^{\text{rot}}_3}})$ </p> 
$\frac{d\ln(q_{\text{rot}})}{dT} = \frac{3}{2T}$ </p> </br>

In [190]:
# Calculates the temp. derivative of the ln of the rotational partition function for one cluster

def calculate_dlnqrot(dlnqrot, temp, atom, linear):
    # lnq     : input/output  - ln of the rotational partition function
    # temp    : input         - temperature
    # atom    : input         - boolean, true if cluster is atom
    # linear  : input         - boolean, true if cluster is linear
    
    
    # Case 1: Cluster is an atom
    if (atom):
        dlnqrot = 0.0
    # Case 2: Cluster is not an atom
    else:
        # Case 2.1: Linear cluster
        # The inertia array contains two equal moments of inertia
        # The square root is taken to account for the degeneracy of the rotational levels
        if (linear):
            dlnqrot = 1.0/temp
            
        # Case 2.2: Non-linear cluster
        # The inertia array contains three moments of inertia
        else:
            dlnqrot = 3.0/(2.0*temp)
            
    return dlnqrot
            

#### Tests

In [187]:
# Case 1: Cluster is an atom
dlnqrot = 0.0
temp = 512.00
atom = True
linear = False
dlnqrot = calculate_dlnqrot(dlnqrot, temp, atom, linear)
print('result',dlnqrot)

result 0.0


In [188]:
# Case 2.1: Linear cluster
dlnqrot = 0.0
temp = 512.00
atom = False
linear = True
dlnqrot = calculate_dlnqrot(dlnqrot, temp, atom, linear)
print('result',dlnqrot)

result 0.001953125


In [189]:
# Case 2.2: Non-linear cluster
dlnqrot = 0.0
temp = 512.00
atom = False
linear = False
dlnqrot = calculate_dlnqrot(dlnqrot, temp, atom, linear)
print('result',dlnqrot)

result 0.0029296875


### Electronic Partition Function Derivative (calculate_dlnqelec)

$\ln(q_{\text{elec}}) = \ln(g) - \frac{E_{\text{elec}}}{kT}$ </p>
$\frac{d\ln(q_{\text{elec}})}{dT} = \frac{E_{\text{elec}}}{kT^2}$ </p>

In [191]:
# Calculates the temp. derivative of the ln of the electronic partition function for one cluster

def calculate_dlnqelec(dlnqelec, temp, energy):
    # lnq  : input/output  - ln of the electronic partition function
    # temp : input         - temperature
    
    dlnqelec = 1.0/avogadro * energy/(kb*temp**2) * 1000.0 # kJ/mol to J/mol (energy is in kJ/mol)
    return dlnqelec

#### Tests

In [193]:
dlnqelec = 0.0
temp = 318.15
energy = -68.23

dlnqelec = calculate_dlnqelec(dlnqelec, temp, energy)
print('result',dlnqelec)

result -0.08107320744165505


In [192]:
dlnqelec = 0.0
temp = 318.15
energy = -28.18

dlnqelec = calculate_dlnqelec(dlnqelec, temp, energy)
print('result',dlnqelec)

result -0.03348443478976754


# Second temp. derivatives of the partition functions

### Vibrational Partition Function second Derivative (calculate_ddlnqvib)

$\dfrac{n\cdot\left(\left(2x\sinh^4\left(\frac{n}{2T}\right)-4\cosh\left(\frac{n}{2T}\right)\sinh^3\left(\frac{n}{2T}\right)+4x\sinh^2\left(\frac{n}{2T}\right)\right)T^2+\left(-n\sinh^4\left(\frac{n}{2T}\right)+n\cosh^2\left(\frac{n}{2T}\right)\sinh^2\left(\frac{n}{2T}\right)-8nx\cosh\left(\frac{n}{2T}\right)\sinh\left(\frac{n}{2T}\right)\right)T-n^2x\sinh^2\left(\frac{n}{2T}\right)+3n^2x\cosh^2\left(\frac{n}{2T}\right)\right)}{4\sinh^4\left(\frac{n}{2T}\right)\,T^5}$